# Test de connexion à la base de données

In [15]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# Informations de connexion à la base de données
host = 'postgres_dst' 
port = '5432'
database = 'dst_airlines_db'
user = 'dst_reader'
password = 'pass_reader'

# Connexion à la base de données
try:
    engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')
    # Charger les données de la table dans un DataFrame pandas
    query_flight_info = 'SELECT * FROM l2.operations_customer_flight_info'  # Requête SQL explicite avec schéma et table
    df = pd.read_sql_query(query_flight_info, con=engine)
    print("Connexion réussie et données flight_info chargées dans le DataFrame !")
    query_airports = 'SELECT * FROM l3.view_airports'
    view_airports = pd.read_sql_query(query_airports, con=engine)
    print("Connexion réussie et données airports chargées dans le DataFrame !")
    query_bird_strikes = 'SELECT * FROM l3.bird_strikes'
    bird_strikes = pd.read_sql_query(query_bird_strikes, con=engine)
    print("Connexion réussie et données bird_strikes chargées dans le DataFrame !")
except Exception as e:
    print("Erreur lors de la connexion ou du chargement des données :", e)



# Afficher les premières lignes du DataFrame pour vérifier
df.head()

Connexion réussie et données flight_info chargées dans le DataFrame !
Connexion réussie et données airports chargées dans le DataFrame !
Connexion réussie et données bird_strikes chargées dans le DataFrame !


,id,departure_airport_code,departure_scheduled_datetime,departure_estimated_datetime,departure_actual_datetime,departure_terminal_name,departure_terminal_gate,departure_status_code,departure_status_description,arrival_airport_code,...,arrival_actual_datetime,arrival_terminal_name,arrival_terminal_gate,arrival_status_code,arrival_status_description,operating_airline_id,operating_flight_nb,equipment_aircraft_code,overall_status_code,overall_status_description
0,95011,BRU,2024-07-19 10:50:00,NaT,2024-07-19 10:59:00,None,B33,DP,Flight Departed,LHR,...,2024-07-19 10:25:00,None,None,LD,Flight Landed,SN,2093,320,LD,Flight Landed
1,95012,DUS,2024-07-19 07:50:00,NaT,2024-07-19 08:06:00,None,A49,DP,Flight Departed,ATH,...,2024-07-19 12:57:00,None,None,LD,Flight Landed,EW,9680,32N,LD,Flight Landed
2,93703,CDG,2024-07-06 10:45:00,NaT,NaT,1,6068,NO,No Status,ZRH,...,NaT,None,None,NO,No Status,2L,633,E90,CD,Flight Cancelled
3,93704,ZRH,2024-07-06 18:35:00,NaT,NaT,None,A65,NO,No Status,AMS,...,NaT,None,None,NO,No Status,LX,734,223,CD,Flight Cancelled
4,93705,AMS,2024-07-06 21:05:00,NaT,NaT,None,None,NO,No Status,ZRH,...,NaT,None,None,NO,No Status,LX,735,223,CD,Flight Cancelled


# Décompte du nombre de portes et de terminaux par aéroports

In [16]:
import pandas as pd

# Filtrer les lignes avec des codes d'aéroports et des gates valides au départ
departures_gate = df[['departure_airport_code', 'departure_terminal_gate']].dropna()
departures_gate = departures_gate[departures_gate['departure_terminal_gate'] != '']

# Filtrer les lignes avec des codes d'aéroports et des gates valides à l'arrivée
arrivals_gate = df[['arrival_airport_code', 'arrival_terminal_gate']].dropna()
arrivals_gate = arrivals_gate[arrivals_gate['arrival_terminal_gate'] != '']

# Renommer les colonnes pour concaténer facilement
departures_gate = departures_gate.rename(columns={'departure_airport_code': 'airport_code', 'departure_terminal_gate': 'gate'})
arrivals_gate = arrivals_gate.rename(columns={'arrival_airport_code': 'airport_code', 'arrival_terminal_gate': 'gate'})

# Concaténer les départs et arrivées pour obtenir un DataFrame unique pour les gates
all_gates = pd.concat([departures_gate[['airport_code', 'gate']], arrivals_gate[['airport_code', 'gate']]])
unique_gates_per_airport = all_gates.drop_duplicates()

# Compter le nombre de gates uniques par aéroport
nb_gates_per_airport = unique_gates_per_airport.groupby('airport_code')['gate'].nunique().reset_index()
nb_gates_per_airport = nb_gates_per_airport.rename(columns={'gate': 'nb_gates'})

# Filtrer les lignes avec des codes d'aéroports et des terminaux valides au départ
departures_terminal = df[['departure_airport_code', 'departure_terminal_name']].dropna()
departures_terminal = departures_terminal[departures_terminal['departure_terminal_name'] != '']

# Filtrer les lignes avec des codes d'aéroports et des terminaux valides à l'arrivée
arrivals_terminal = df[['arrival_airport_code', 'arrival_terminal_name']].dropna()
arrivals_terminal = arrivals_terminal[arrivals_terminal['arrival_terminal_name'] != '']

# Renommer les colonnes pour concaténer facilement
departures_terminal = departures_terminal.rename(columns={'departure_airport_code': 'airport_code', 'departure_terminal_name': 'terminal'})
arrivals_terminal = arrivals_terminal.rename(columns={'arrival_airport_code': 'airport_code', 'arrival_terminal_name': 'terminal'})

# Concaténer les départs et arrivées pour obtenir un DataFrame unique pour les terminaux
all_terminals = pd.concat([departures_terminal[['airport_code', 'terminal']], arrivals_terminal[['airport_code', 'terminal']]])
unique_terminals_per_airport = all_terminals.drop_duplicates()

# Compter le nombre de terminaux uniques par aéroport
nb_terminals_per_airport = unique_terminals_per_airport.groupby('airport_code')['terminal'].nunique().reset_index()
nb_terminals_per_airport = nb_terminals_per_airport.rename(columns={'terminal': 'nb_terminals'})

# Fusionner les deux DataFrames sur la colonne airport_code
result = pd.merge(nb_terminals_per_airport, nb_gates_per_airport, on='airport_code', how='outer')

# Calculer le ratio gates_per_terminal quand nb_terminals et nb_gates sont valides
result['gates_per_terminal'] = result.apply(
    lambda row: row['nb_gates'] / row['nb_terminals'] if pd.notnull(row['nb_gates']) and pd.notnull(row['nb_terminals']) and row['nb_terminals'] > 0 else None,
    axis=1
)

# Extraire les dates de décollage les plus anciennes et les plus récentes
# Filtrer les données avec des dates de décollage valides
valid_departures = df[['departure_airport_code', 'departure_actual_datetime']].dropna()
valid_departures = valid_departures.rename(columns={'departure_airport_code': 'airport_code'})

# Calculer la date de décollage la plus ancienne et la plus récente pour chaque aéroport
departure_dates = valid_departures.groupby('airport_code')['departure_actual_datetime'].agg(
    oldest_departure='min', latest_departure='max'
).reset_index()

# Fusionner les informations de date avec le DataFrame result
result = pd.merge(result, departure_dates, on='airport_code', how='left')

# Calculer les statistiques des vols de départ et d'arrivée
# Remplir les valeurs pour chaque statut dans departure_status_code
departure_stats = df.groupby('departure_airport_code')['departure_status_code'].value_counts().unstack(fill_value=0).reset_index()
departure_stats = departure_stats.rename(columns={
    'departure_airport_code': 'airport_code',
    'DL': 'nb_departure_flight_delayed',
    'DP': 'nb_departure_flight_departed',
    'OT': 'nb_departure_flight_on_time'
})
departure_stats['nb_departure_flight_total'] = departure_stats.iloc[:, 1:].sum(axis=1)
departure_stats['nb_departure_flight_no_status'] = departure_stats['nb_departure_flight_total'] - (
    departure_stats.get('nb_departure_flight_delayed', 0) + 
    departure_stats.get('nb_departure_flight_departed', 0) + 
    departure_stats.get('nb_departure_flight_on_time', 0)
)

# Supprimer la colonne 'NO' si elle existe dans departure_stats
if 'NO' in departure_stats.columns:
    departure_stats = departure_stats.drop(columns=['NO'])

# Remplir les valeurs pour chaque statut dans overall_status_code
arrival_stats = df.groupby('arrival_airport_code')['overall_status_code'].value_counts().unstack(fill_value=0).reset_index()
arrival_stats = arrival_stats.rename(columns={
    'arrival_airport_code': 'airport_code',
    'OT': 'nb_overall_flight_on_time',
    'FE': 'nb_overall_flight_early',
    'CD': 'nb_overall_flight_cancelled',
    'DL': 'nb_overall_flight_delayed',
    'DV': 'nb_overall_flight_diverted',
    'LD': 'nb_overall_flight_landed'
})
arrival_stats['nb_overall_flight_total'] = arrival_stats.iloc[:, 1:].sum(axis=1)
arrival_stats['nb_overall_no_status'] = arrival_stats['nb_overall_flight_total'] - (
    arrival_stats.get('nb_overall_flight_on_time', 0) +
    arrival_stats.get('nb_overall_flight_early', 0) + 
    arrival_stats.get('nb_overall_flight_cancelled', 0) + 
    arrival_stats.get('nb_overall_flight_delayed', 0) + 
    arrival_stats.get('nb_overall_flight_diverted', 0) + 
    arrival_stats.get('nb_overall_flight_landed', 0)
)

# Fusionner les statistiques des départs et des arrivées avec le DataFrame result
result = pd.merge(result, departure_stats, on='airport_code', how='left')
result = pd.merge(result, arrival_stats, on='airport_code', how='left')


# Renommer AirportCode en airport_code pour correspondre à la clé de jointure avec result
view_airports = view_airports.rename(columns={
    'AirportCode': 'airport_code',
    'AirportNameFR': 'airport_name_FR',
    'AirportNameEN': 'airport_name_EN',
    'AirportLatitude': 'airport_latitude',
    'AirportLongitude': 'airport_longitude'
})

# Fusionner view_airports avec result sur airport_code
result = pd.merge(result, view_airports[['airport_code', 'airport_name_FR', 'airport_name_EN', 'airport_latitude', 'airport_longitude']], 
                  on='airport_code', how='left')

# Compter les occurrences de chaque type d'événement par aéroport
bird_strike_counts = bird_strikes.groupby(['airport_code', 'event_tag']).size().unstack(fill_value=0).reset_index()

# Renommer les colonnes selon le format demandé
bird_strike_counts = bird_strike_counts.rename(columns={
    'Accident': 'nb_bird_event_accident',
    'Incident': 'nb_bird_event_incident',
    'Report': 'nb_bird_event_report',
    'Crash': 'nb_bird_event_crash'
})

# Fusionner les données de bird_strike_counts avec le DataFrame result sur la colonne airport_code
result = pd.merge(result, bird_strike_counts[['airport_code', 'nb_bird_event_accident', 'nb_bird_event_incident', 'nb_bird_event_report', 'nb_bird_event_crash']], 
                  on='airport_code', how='left')

# Remplacer les valeurs NaN résultant de la fusion par 0, pour indiquer l'absence d'événements pour ces types
result[['nb_bird_event_accident', 'nb_bird_event_incident', 'nb_bird_event_report', 'nb_bird_event_crash']] = result[['nb_bird_event_accident', 'nb_bird_event_incident', 'nb_bird_event_report', 'nb_bird_event_crash']].fillna(0)


# Afficher le résultat final
print(result)



    airport_code  nb_terminals  nb_gates  gates_per_terminal  \
0            ABV           1.0       2.0                 2.0   
1            ACE           1.0       NaN                 NaN   
2            AGP           NaN      11.0                 NaN   
3            ALA           1.0       2.0                 2.0   
4            ALC           NaN       5.0                 NaN   
..           ...           ...       ...                 ...   
201          YYZ           1.0       6.0                 6.0   
202          ZAG           NaN       7.0                 NaN   
203          ZNZ           1.0       NaN                 NaN   
204          ZRH           NaN      90.0                 NaN   
205          ZTH           NaN       1.0                 NaN   

       oldest_departure    latest_departure  nb_departure_flight_delayed  \
0   2024-07-06 23:14:00 2024-10-30 00:15:00                          0.0   
1   2024-07-07 17:51:00 2024-10-29 16:20:00                          0.0   
2  

## Top 10 des aéroports avec le plus de gates

In [17]:
# Afficher les 10 aéroports avec le plus grand nombre de terminaux uniques
top_10_airports_gates = result.sort_values('nb_gates', ascending=False).head(10)

print("Top 10 des aéroports avec le plus de portes uniques :")
print(top_10_airports_gates)

Top 10 des aéroports avec le plus de portes uniques :
    airport_code  nb_terminals  nb_gates  gates_per_terminal  \
128          MUC           2.0     190.0           95.000000   
68           FRA           1.0     167.0          167.000000   
192          VIE           2.0     116.0           58.000000   
57           DUS           NaN      90.0                 NaN   
204          ZRH           NaN      90.0                 NaN   
17           BER           1.0      83.0           83.000000   
34           BRU           NaN      73.0                 NaN   
64           FCO           1.0      51.0           51.000000   
175          STR           3.0      47.0           15.666667   
80           HAM           2.0      43.0           21.500000   

       oldest_departure    latest_departure  nb_departure_flight_delayed  \
128 2024-07-05 13:45:00 2024-10-31 00:01:00                          0.0   
68  2024-07-05 15:31:00 2024-10-30 23:30:00                          3.0   
192 2024-07-0

## Détails des gates pour un aéroports

In [21]:
# Filtrer pour ne garder que les lignes où airport_code est "MUC"
muc_gates = unique_gates_per_airport[unique_gates_per_airport['airport_code'] == 'MUC']

# Afficher les terminaux uniques pour l'aéroport "MUC"
print("Portes pour l'aéroport MUC :")
print(muc_gates['gate'].unique())

# Filtrer pour ne garder que les lignes où airport_code est "CDG"
cdg_gates = unique_gates_per_airport[unique_gates_per_airport['airport_code'] == 'CDG']

# Afficher les terminaux uniques pour l'aéroport "MUC"
print("Portes pour l'aéroport CDG :")
print(cdg_gates['gate'].unique())

Portes pour l'aéroport MUC :
['G18' 'H34' 'H18' 'G34' 'G24' 'K27' 'G28' 'G35' 'L11' 'G64' 'G82' 'G27'
 'G33' 'K16' 'G71' 'G47' 'G65' 'G31' 'G41' 'K22' 'G63' 'G81' 'G17' 'K08'
 'L13' 'K12' 'L15' 'G19' 'G11' 'G66' 'G69' 'G30' 'K03' 'K14' 'D12' 'K18'
 'G29' 'K04' 'H40' 'A16' 'A21' 'G72' 'G70' 'K07' 'G20' 'A26' 'G08' 'A14'
 'G40' 'G37' 'H38' 'D06' 'L21' 'H44' 'G44' 'H46' 'G39' 'H30' 'L10' 'H32'
 'K30' 'G48' 'G10' 'H37' 'G32' 'K09' 'L30' 'H16' 'G26' 'L26' 'H26' 'H39'
 'L01' 'G07' 'L03' 'L06' 'K05' 'K10' 'G16' 'G14' 'K25' 'H14' 'H20' 'K19'
 'K21' 'L19' 'H24' 'G46' 'H35' 'H48' 'H33' 'G02' 'H19' 'K26' 'H28' 'H08'
 'G38' 'K15' 'H12' 'K01' 'L09' 'K20' 'L25' 'K23' 'L08' 'H45' 'K11' 'K13'
 'G04' 'L22' 'K06' 'G23' 'L07' 'G15' 'G06' 'G84' 'K24' 'G13' 'G43' 'G01'
 'L27' 'G03' 'F21' 'H02' 'L04' 'K28' 'L28' 'H41' 'H04' 'H15' 'L05' 'G05'
 'G09' 'L24' 'G36' 'H27' 'G45' 'L23' 'G42' 'G22' 'H29' 'H47' 'G12' 'H06'
 'H42' 'H05' 'H01' 'H23' 'H17' 'H10' 'H31' 'D15' 'D03' 'D07' 'D04' 'D11'
 'D01' 'T02N' 'T02S' '

## Top 10 des aéroports avec le plus de terminaux

In [24]:
# Afficher les 10 aéroports avec le plus grand nombre de terminaux uniques
top_10_airports_term = result.sort_values('nb_terminals', ascending=False).head(10)

print("Top 10 des aéroports avec le plus de terminaux uniques :")
print(top_10_airports_term)

Top 10 des aéroports avec le plus de terminaux uniques :
    airport_code  nb_terminals  nb_gates
175          STR           3.0      47.0
62           EZE           2.0      10.0
36           BUD           2.0       7.0
38           CAI           2.0       7.0
131          NBO           2.0       1.0
85           HRG           2.0       NaN
121          MAN           2.0      27.0
192          VIE           2.0     116.0
152          PUJ           2.0       NaN
113          LIS           2.0      18.0


## Détails des terminaux pour un aéroports

In [25]:
# Filtrer pour ne garder que les lignes où airport_code est "MUC"
muc_terms = unique_terminals_per_airport[unique_terminals_per_airport['airport_code'] == 'MUC']

# Afficher les terminaux uniques pour l'aéroport "MUC"
print("Terminaux pour l'aéroport MUC :")
print(muc_terms['terminal'].unique())

# Filtrer pour ne garder que les lignes où airport_code est "CDG"
cdg_terms = unique_terminals_per_airport[unique_terminals_per_airport['airport_code'] == 'CDG']

# Afficher les terminaux uniques pour l'aéroport "MUC"
print("Terminaux pour l'aéroport CDG :")
print(cdg_terms['terminal'].unique())

Terminaux pour l'aéroport MUC :
['2' '1']
Terminaux pour l'aéroport CDG :
['1']


## Top 10 des aéroports avec le plus d'accidents liés aux oiseaux

In [18]:
# Afficher les 10 aéroports avec le plus grand nombre de d'accidents liés aux oiseaux
top_10_birds_accidents = result.sort_values('nb_bird_event_accident', ascending=False).head(10)

print("Top 10 des aéroports qui présentent le plus grand nombre d'accidents liés aux oiseaux :")
print(top_10_birds_accidents)

Top 10 des aéroports qui présentent le plus grand nombre d'accidents liés aux oiseaux :
    airport_code  nb_terminals  nb_gates  gates_per_terminal  \
7            AMS           NaN      31.0                 NaN   
11           ATL           1.0       3.0                 3.0   
46           CPT           NaN       7.0                 NaN   
61           EWR           1.0       8.0                 8.0   
45           CPH           1.0      14.0                14.0   
27           BLQ           NaN       3.0                 NaN   
120          MAD           1.0      30.0                30.0   
131          NBO           2.0       1.0                 0.5   
92           IKA           NaN       2.0                 NaN   
80           HAM           2.0      43.0                21.5   

       oldest_departure    latest_departure  nb_departure_flight_delayed  \
7   2024-07-06 08:06:00 2024-10-30 21:25:00                          0.0   
11  2024-07-06 12:08:00 2024-10-30 12:41:00            

## Top 10 des aéroports avec le plus de vols annulés

In [19]:
# Afficher les 10 aéroports avec le plus grand nombre de vols annulés
top_10_flight_cancelled = result.sort_values('nb_overall_flight_cancelled', ascending=False).head(10)

print("Top 10 des aéroports qui présentent le plus grand nombre de vols annulés :")
print(top_10_flight_cancelled)



Top 10 des aéroports qui présentent le plus grand nombre de vols annulés :
    airport_code  nb_terminals  nb_gates  gates_per_terminal  \
68           FRA           1.0     167.0               167.0   
17           BER           1.0      83.0                83.0   
128          MUC           2.0     190.0                95.0   
204          ZRH           NaN      90.0                 NaN   
111          LHR           1.0      25.0                25.0   
192          VIE           2.0     116.0                58.0   
7            AMS           NaN      31.0                 NaN   
132          NCE           1.0      15.0                15.0   
57           DUS           NaN      90.0                 NaN   
39           CDG           1.0      14.0                14.0   

       oldest_departure    latest_departure  nb_departure_flight_delayed  \
68  2024-07-05 15:31:00 2024-10-30 23:30:00                          3.0   
17  2024-07-06 06:58:00 2024-10-30 22:23:00                         